## This notebook can be used to prepare the data  used for classifying invasive species
#### Input: Reference points (GEE FeatureCollection) containing the classes of interest as a property.
##### Arguments: A start and end data, cloud threshold, area of interest, image composite interval
#### Output: given dates, cloud threshold and an aoi, create invasive map

## Step 1: Load modules

In [14]:
%load_ext watermark

In [15]:
# Load python modue with preprocessing functions. 
import ee
# ee.Authenticate()
ee.Initialize()

# Add module to environment varibles
import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\python_modules'
sys.path.insert(1, MODULE_FULL_PATH)

import geemap
from TSfunctions import prepareTS
from covariateFunctions import prepareCovariates
from trainDataFunctions import prepareTrainingData
from geeml.utils import eeprint

In [16]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.5.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

geemap: 0.19.6
sys   : 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
ee    : 0.1.327



In [4]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Step 2: Prepare and Export data

### Prepare Sentinel-2 Data

In [17]:
# start and end dates for time series
Date_Start = ee.Date('2017-01-01');
Date_End = ee.Date('2017-12-31');

# how many days to summarise in each image e.g 30 days = 12 images per year
day_int = 30 #step size

# cloud probabillity threshold
CLOUD_THRESH=30

# which bands to keep
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6','B7', 'B8', 'B8A', 'B9', 'B11','B12', 'ndvi','ndwi','ndre','nbr','evi']

aoi = ee.Geometry.Point([25.181, -28.490])
# define your study area
study_area1 = ee.Geometry.Polygon([[[30.555572509765607, -24.407014032416797],
          [31.065750122070295, -24.40888986482876],
          [31.065750122070295, -24.40888986482876],
          [31.049957275390607, -25.121539326337704],
          [30.560379028320295, -25.133972597740186],
          [30.555572509765607, -24.407014032416797]]], None, False)

# flatten imagecollection to image - each image becomes a band
s2TS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDS)\
.s2TimeSeries()

# Add additional terrain related covariates
X = prepareCovariates('EPSG:32726').addTopoBands(s2TS)

In [6]:
Map.addLayer(study_area1)
Map.centerObject(atudy_area1, 8)

## Step 3: Prepare data

### Prepare covariates (added) and training dataset.

#### Load training data

In [11]:
# Import reference points
points = ee.FeatureCollection("users/moilwekk/merged_classes")

# name of the label property
label = 'code'

# number of folds for kfold
FOLDS = 10       

# Extract covariate image values to points
pTrainData = prepareTrainingData(s2TS, points, label, FOLDS, 10, 'EPSG:32726')
train = pTrainData.covariatesToPoints()
train.limit(5)

## Step 4: Export data

### Export training data and image to run inference on

In [13]:
# export training data and image to classify
# Define the export parameters
export_params1 = {
    'collection': train,
    'description': 'export_trainData',
    'assetId': 'users/geethensingh/trainData'
}

export_params2 = {
    'image': s2TS,
    'description': 'export_inferenceImage',
    'assetId': 'users/geethensingh/inferenceImage',
    'scale': 10,
    'region': study_area1
}

# Export the data to the Earth Engine asset
ee.batch.Export.table.toAsset(**export_params1).start()
ee.batch.Export.image.toAsset(**export_params2).start()